In [1]:
import os, sys
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import datetime
import time

# df_db와 df_inout, df_crd 연결

In [115]:
df_db = pd.read_csv(f"./final_db/df_db_hday_rvs.csv", index_col=[0])
#df_inout = pd.read_csv("./final_db/2015_subway_day_hour_1234.csv", index_col=[0])
df_crd = pd.read_csv('./database/crowded2015.csv', index_col=[0])

/Users/bowoojang/Desktop/idk/venv/univ_env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [129]:
df_db = df_db.drop_duplicates()
df_crd = df_crd.drop_duplicates()

df_crd = df_crd.groupby(by=['Day', 'Line', 'Direction', 'Station']).mean()
df_crd = df_crd.reset_index(drop=False)

In [134]:
# 2호선 내선/외선 이외의 분기 노선 모두 삭제 
line2_ex_list = ['까치산', '신정네거리', '양천구청', '도림천', '용답', '신답', '용두', '신설동']
cond = df_crd.loc[:, 'Station'].apply(lambda x: x in line2_ex_list)

drop_idx = df_crd.loc[(df_crd['Line']=='2호선') & cond, :].index
df_crd = df_crd.drop(index=drop_idx)
df_crd = df_crd.reset_index(drop=True)

df_crd.columns = ['holiday', 'line', 'order', 'stt', ] + list(df_crd.columns)[-20:]

def holiday(x):
    if x == "평일":
        return 0
    elif x == "토요일":
        return 1
    else:
        return 2
    
df_crd['holiday'] = df_crd['holiday'].apply(lambda x: holiday(x))
df_crd


,holiday,line,order,stt,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,2,1호선,상선,동대문,6.0,10.0,15.0,17.0,21.0,24.0,...,46.0,44.0,45.0,36.0,30.0,29.0,32.0,30.0,18.0,16.0
1,2,1호선,상선,동묘앞,4.0,10.0,14.0,15.0,21.0,23.0,...,42.0,45.0,43.0,35.0,28.0,26.0,32.0,33.0,20.0,39.0
2,2,1호선,상선,서울역,4.0,12.0,17.0,28.0,31.0,37.0,...,51.0,47.0,41.0,33.0,26.0,23.0,23.0,22.0,13.0,19.0
3,2,1호선,상선,시청,5.0,12.0,16.0,24.0,28.0,35.0,...,50.0,48.0,42.0,33.0,28.0,26.0,26.0,25.0,15.0,22.0
4,2,1호선,상선,신설동,5.0,10.0,14.0,15.0,20.0,22.0,...,40.0,47.0,42.0,36.0,27.0,26.0,29.0,31.0,19.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,0,4호선,하선,총신대입구,0.0,23.0,28.0,48.0,33.0,33.0,...,49.0,50.0,52.0,66.0,55.0,43.0,50.0,52.0,45.0,16.0
674,0,4호선,하선,충무로,34.0,24.0,48.0,76.0,60.0,47.0,...,57.0,52.0,47.0,51.0,36.0,32.0,35.0,41.0,32.0,9.0
675,0,4호선,하선,한성대입구,46.0,44.0,110.0,156.0,108.0,70.0,...,51.0,46.0,40.0,40.0,27.0,26.0,24.0,26.0,24.0,8.0
676,0,4호선,하선,혜화,47.0,43.0,100.0,154.0,107.0,74.0,...,62.0,58.0,50.0,52.0,34.0,36.0,39.0,50.0,40.0,12.0


In [138]:
st_t = time.time()
for line in df_db['line'].unique():
    cond_db1 = df_db['line']==line
    cond_crd1 = df_crd['line']==line
    
    for order in df_db['order'].unique():
        cond_db2 = df_db['order']==order
        cond_crd2 = df_crd['order']==order
        
        for holiday in df_db['holiday'].unique():
            cond_db3 = df_db['holiday']== holiday
            cond_crd3 = df_crd['holiday']== holiday
            
            for stt in df_db.loc[cond_db1&cond_db2, 'from_stt'].unique():
                cond_db4 = df_db['from_stt']== stt
                cond_crd4 = df_crd['stt'] == stt
                
                try:
                    N = (df_db.loc[cond_db1&cond_db2&cond_db3&cond_db4, :].shape[0])//20
                    print(df_db.loc[cond_db1&cond_db2&cond_db3&cond_db4, :].shape)
                    copy = df_crd.loc[cond_crd1&cond_crd2&cond_crd3&cond_crd4, '5':].to_numpy().reshape(-1,1)
                    
                    print(copy.shape, N)
                    copy = np.vstack([copy]*N)
                    df_db.loc[cond_db1&cond_db2&cond_db3&cond_db4, "crowd"] = copy
                except:
                    print(df_db.loc[cond_db1&cond_db2&cond_db3&cond_db4, :])
                    print(df_crd.loc[cond_crd1&cond_crd2&cond_crd3&cond_crd4,:])
                    print(copy, N)
                 
        print(f"Order {order} end: {time.time() - st_t}")
    print(f"Line no.{line} end: {time.time() - st_t}")

(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
Order 상선 end: 3.73978590965271
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(1320, 10)
(20, 1) 66
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 

(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(5020, 10)
(20, 1) 251
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 10)
(20, 1) 48
(960, 

In [144]:
df_db.to_csv(f"./final_db/df_db_hday_rvs_crd.csv")

In [145]:
df_db

,holiday,date,time,line,order,from_stt,to_stt,in,out,crowd
0,2,2015-01-01,5,1호선,상선,서울역,시청,NaN,NaN,4.0
1,2,2015-01-01,5,1호선,상선,시청,종각,NaN,NaN,5.0
2,2,2015-01-01,5,1호선,상선,종각,종로3가,NaN,NaN,6.0
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,NaN,NaN,6.0
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...
1605995,0,2015-12-31,24,4호선,하선,쌍문,수유,NaN,NaN,7.0
1605996,0,2015-12-31,24,4호선,하선,창동,쌍문,NaN,NaN,8.0
1605997,0,2015-12-31,24,4호선,하선,노원,창동,NaN,NaN,7.0
1605998,0,2015-12-31,24,4호선,하선,상계,노원,NaN,NaN,2.0


crd, inout 파일 합치기

In [154]:
df_db_inout = pd.read_csv(f"./final_db/df_db_hday_rvs_inout.csv", index_col=[0])
df_db_inout = df_db_inout.drop(columns=['cong', 'crowd'])

/Users/bowoojang/Desktop/idk/venv/univ_env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [156]:
df_db_inout

,holiday,date,time,line,order,from_stt,to_stt,in,out
0,2,2015-01-01,5,1호선,상선,서울역,시청,441.0,392.0
1,2,2015-01-01,5,1호선,상선,시청,종각,101.0,57.0
2,2,2015-01-01,5,1호선,상선,종각,종로3가,898.0,60.0
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,371.0,27.0
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,57.0,22.0
...,...,...,...,...,...,...,...,...,...
1605995,0,2015-12-31,24,4호선,하선,쌍문,수유,197.0,1128.0
1605996,0,2015-12-31,24,4호선,하선,창동,쌍문,200.0,987.0
1605997,0,2015-12-31,24,4호선,하선,노원,창동,346.0,1104.0
1605998,0,2015-12-31,24,4호선,하선,상계,노원,116.0,717.0


In [151]:
df_db_crd = df_db.copy()
df_db_crd = df_db_crd.drop(columns=['in', 'out'])

In [158]:
df_db_crd

,holiday,date,time,line,order,from_stt,to_stt,crowd
0,2,2015-01-01,5,1호선,상선,서울역,시청,4.0
1,2,2015-01-01,5,1호선,상선,시청,종각,5.0
2,2,2015-01-01,5,1호선,상선,종각,종로3가,6.0
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,6.0
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,6.0
...,...,...,...,...,...,...,...,...
1605995,0,2015-12-31,24,4호선,하선,쌍문,수유,7.0
1605996,0,2015-12-31,24,4호선,하선,창동,쌍문,8.0
1605997,0,2015-12-31,24,4호선,하선,노원,창동,7.0
1605998,0,2015-12-31,24,4호선,하선,상계,노원,2.0


In [160]:
df_db_inout.merge(df_db_crd, how='outer').to_csv("./final_db/data.csv")

In [161]:
df_db_inout.merge(df_db_crd, how='outer')

,holiday,date,time,line,order,from_stt,to_stt,in,out,crowd
0,2,2015-01-01,5,1호선,상선,서울역,시청,441.0,392.0,4.0
1,2,2015-01-01,5,1호선,상선,시청,종각,101.0,57.0,5.0
2,2,2015-01-01,5,1호선,상선,종각,종로3가,898.0,60.0,6.0
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,371.0,27.0,6.0
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,57.0,22.0,6.0
...,...,...,...,...,...,...,...,...,...,...
1605995,0,2015-12-31,24,4호선,하선,쌍문,수유,197.0,1128.0,7.0
1605996,0,2015-12-31,24,4호선,하선,창동,쌍문,200.0,987.0,8.0
1605997,0,2015-12-31,24,4호선,하선,노원,창동,346.0,1104.0,7.0
1605998,0,2015-12-31,24,4호선,하선,상계,노원,116.0,717.0,2.0
